<h3> Purpose: In this notebook, we get the raw data and conduct data processing <h3>

<h4> Data: Data is collected from FRED - the federal reserve economic data for this project. The values returned are the single "level" value per day and we will treat is like closing prices for modelling purposes. The portfolio consists of SPY (Proxy for S&P 500) and EUR/USD (major FX pair) to represent equity and FX risk respectively. These instruments are highly liquid and commonly used as market proxies, allowing us to focus on model risk rather than data quality issues <h4>

<h5> <b>Liquidity</b> = assumptions hold better as risk models assume things like: 
- prices reflect information quickly
- no artificial price jumps due to illiquidity
- historical prices are meaningful <h6>

<h5> <b>Widely used assets</b> = benchmarkability as real validation work asks "does this risk number look reasonable" and "how does this compare to industry norms". With SPY and EUR/USD, crisis behavious are well documented and stress periods are identifiable <h6>

<h5> <b>Easy to justify</b> = time is spent on model logic, not data choise <h6>

<h4> Time horizon: daily data & 2013-2024 <h4>

In [43]:
# Import libraries
import yfinance as yf
import pandas as pd
print(pd.__version__)
import warnings
import numpy as np
from typing import List
from pandas_datareader import data as pdr
warnings.filterwarnings("ignore", category=Warning)

2.3.3


<h3> 1. Import Data

In [45]:
start = "2013-01-01"
end = "2024-12-31"

sp500 = pdr.DataReader("SP500", "fred", start, end) #sp500
t10y = pdr.DataReader("GS10", "fred", start, end)  # 10-year Treasury yield
eurusd = pdr.DataReader("DEXUSEU", "fred", start, end) #FX rate EUR/USD

In [46]:
print(type(sp500))
print(sp500.describe())
print(sp500.head())

print(type(t10y))
print(t10y.describe())
print(t10y.head())

print(type(eurusd))
print(eurusd.describe())
print(eurusd.head())

<class 'pandas.core.frame.DataFrame'>
             SP500
count  2250.000000
mean   3510.076333
std    1043.581806
min    1829.080000
25%    2664.020000
50%    3291.950000
75%    4313.075000
max    6090.270000
              SP500
DATE               
2016-01-25  1877.08
2016-01-26  1903.63
2016-01-27  1882.95
2016-01-28  1893.36
2016-01-29  1940.24
<class 'pandas.core.frame.DataFrame'>
             GS10
count  144.000000
mean     2.475625
std      0.968875
min      0.620000
25%      1.810000
50%      2.355000
75%      2.900000
max      4.800000
            GS10
DATE            
2013-01-01  1.91
2013-02-01  1.98
2013-03-01  1.96
2013-04-01  1.76
2013-05-01  1.93
<class 'pandas.core.frame.DataFrame'>
           DEXUSEU
count  2999.000000
mean      1.153941
std       0.093277
min       0.961600
25%       1.088750
50%       1.126200
75%       1.189800
max       1.392700
            DEXUSEU
DATE               
2013-01-01      NaN
2013-01-02   1.3195
2013-01-03   1.3110
2013-01-04   1.3047
201

<h3> 2. Data Cleaning <h3>

1. Align dates across assets as different markets have different holidays
2. Compute log returns: Time additive, better behaved statistically
3. Check for other issues

In [50]:
print(sp500.columns.tolist())
print(sp500.index)

['SP500']
DatetimeIndex(['2016-01-25', '2016-01-26', '2016-01-27', '2016-01-28',
               '2016-01-29', '2016-02-01', '2016-02-02', '2016-02-03',
               '2016-02-04', '2016-02-05',
               ...
               '2024-12-18', '2024-12-19', '2024-12-20', '2024-12-23',
               '2024-12-24', '2024-12-25', '2024-12-26', '2024-12-27',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', name='DATE', length=2332, freq=None)


In [51]:
# Merge on 'DATE' index
data = pd.concat([sp500, eurusd], axis=1, join='inner')  # inner keeps only matching dates
data

,SP500,DEXUSEU
DATE,,
2016-01-25,1877.08,1.0832
2016-01-26,1903.63,1.0846
2016-01-27,1882.95,1.0867
2016-01-28,1893.36,1.0952
2016-01-29,1940.24,1.0832
...,...,...
2024-12-25,NaN,NaN
2024-12-26,6037.59,1.0423
2024-12-27,5970.84,1.0423


In [53]:
print(data.columns.tolist())
print(data.describe())
print("\n number of NA \n", data.isna().sum())


['SP500', 'DEXUSEU']
             SP500      DEXUSEU
count  2250.000000  2233.000000
mean   3510.076333     1.120100
std    1043.581806     0.054199
min    1829.080000     0.961600
25%    2664.020000     1.083600
50%    3291.950000     1.115600
75%    4313.075000     1.162500
max    6090.270000     1.248800

 number of NA 
 SP500      82
DEXUSEU    99
dtype: int64


In [54]:
data = data.dropna()
print(data.describe())
print("\n number of NA \n", data.isna().sum())

             SP500      DEXUSEU
count  2224.000000  2224.000000
mean   3510.012986     1.120110
std    1043.561808     0.054210
min    1829.080000     0.961600
25%    2664.080000     1.083675
50%    3288.710000     1.115600
75%    4310.025000     1.162525
max    6090.270000     1.248800

 number of NA 
 SP500      0
DEXUSEU    0
dtype: int64


<h5> Here, we calculate log returns as it is <h5> 

1. Time additive meaning that multy day returns = sum of daily log returns which aids the ease of VaR calculations over several days. 
2. Log returns are more symmetric and is better for normality assumptions 
3. Most risk models assume returns, not prices

In [57]:
# Calculate Log Returns ln(Pt / Pt-1)
# 0 < x < 1 : negative log returns (when Pt < Pt-1, price decreased)
# x > 1 : positive log returns (when Pt > Pt-1, price increased

log_returns = np.log(data / data.shift(1)).dropna()
log_returns.columns = ['SP500_log', 'EURUSD_log']
log_returns

,SP500_log,EURUSD_log
DATE,,
2016-01-26,0.014045,0.001292
2016-01-27,-0.010923,0.001934
2016-01-28,0.005513,0.007791
2016-01-29,0.024459,-0.011017
2016-02-01,-0.000443,0.005157
...,...,...
2024-12-24,0.010982,-0.002020
2024-12-26,-0.000406,0.003364
2024-12-27,-0.011117,0.000000


<h3> 3. Export cleaned data

In [59]:
log_returns.to_csv("../data/log_returns.csv")